In [1]:
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
import os

# Get the current directory
current_directory = os.getcwd()

# List all files and directories in the current directory
files_and_directories = os.listdir(current_directory)

# Print the list
print("Files and directories in", current_directory)
for item in files_and_directories:
    print(item)


Files and directories in /kaggle/working
.virtual_documents


In [3]:
import tensorflow as tf
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import Concatenate
from keras.layers import Flatten
from keras.layers import Dense
from keras.applications import VGG16, VGG19, ResNet50, ResNet101

2024-05-22 11:49:45.138054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 11:49:45.138151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 11:49:45.284220: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')


BaseException: ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!

In [5]:
print('Running on TPU ', tpu.master())

Running on TPU  


In [5]:
complete_data_info = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")

In [6]:
complete_data_info.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [7]:
data_base_path = "/kaggle/input/data"

In [8]:
complete_data_imgs_paths = list()
imgs_file_names = list()

for single_img_path in Path(data_base_path).glob("images_*/images/*.png"):
    
    complete_data_imgs_paths.append(str(single_img_path))
    imgs_file_names.append(str(single_img_path.parts[-1]))

In [9]:
complete_data_path_info = pd.DataFrame(data={"Image Index":imgs_file_names,
                                            "Image Path":complete_data_imgs_paths})

In [10]:
complete_data_all_info = complete_data_info.merge(complete_data_path_info,on="Image Index")

In [11]:
complete_data_all_info.drop(complete_data_all_info.columns[2:6],axis=1,inplace=True)

In [12]:
complete_data_all_info.drop(["Unnamed: 11"],axis=1,inplace=True)

In [15]:
complete_data_all_info.head()

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
0,00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [16]:
testing_data_info = pd.read_csv("/kaggle/input/data/BBox_List_2017.csv")

In [17]:
testing_data_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN


In [18]:
complete_data_all_info = complete_data_all_info.set_index("Image Index")

In [19]:
testing_data_path_info = complete_data_all_info.loc[testing_data_info["Image Index"]]

In [20]:
training_data_all_info = complete_data_all_info.drop(index=testing_data_path_info.index)

In [21]:
training_data_all_info.head()

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
Image Index,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [22]:
plt.imread(training_data_all_info.iat[2, -1]).shape

(1024, 1024)

In [23]:
testing_data_path_info.reset_index(inplace=True)

In [24]:
testing_data_all_info = testing_data_info.merge(testing_data_path_info,
                                               on="Image Index")

In [25]:
testing_data_all_info = pd.concat([testing_data_info, testing_data_path_info], axis = 1)

In [26]:
testing_data_all_info.shape

(984, 17)

In [27]:
testing_data_all_info.drop(labels=testing_data_all_info.columns[6:9],axis=1,
                          inplace=True)

In [28]:
training_data_all_info["Finding Labels"] = training_data_all_info["Finding Labels"].map(lambda x: x.split("|"))
testing_data_all_info["Finding Labels"] = testing_data_all_info["Finding Labels"].map(lambda x: x.split("|"))

In [29]:
merged_list = list()

for single_list in training_data_all_info["Finding Labels"]:
    
    merged_list = merged_list + single_list

unique_diseases = set(merged_list)
unique_diseases = list(unique_diseases)
unique_diseases.remove("No Finding")
disease2idx = dict(zip(unique_diseases,range(0,len(unique_diseases))))

In [31]:
unique_diseases
disease2idx

{'Infiltration': 0,
 'Pneumonia': 1,
 'Fibrosis': 2,
 'Atelectasis': 3,
 'Pneumothorax': 4,
 'Consolidation': 5,
 'Edema': 6,
 'Pleural_Thickening': 7,
 'Nodule': 8,
 'Emphysema': 9,
 'Cardiomegaly': 10,
 'Hernia': 11,
 'Mass': 12,
 'Effusion': 13}

In [32]:
def map_diseases(disease_list):
    
    all_zeros = np.zeros(len(disease2idx),)
    
    for single_disease in disease_list:
        
        if single_disease != "No Finding":
            all_zeros[disease2idx[single_disease]] = 1
        
    return all_zeros

In [126]:
with ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:
    
    multi_hot_encoded_Y_train = np.array(list(pool.map(map_diseases,training_data_all_info["Finding Labels"])))
    multi_hot_encoded_Y_test = np.array(list(pool.map(map_diseases,testing_data_all_info["Finding Labels"])))

In [127]:
training_data_all_info.reset_index(inplace=True)


In [128]:
training_data_all_info.to_csv("training_data.csv",index=False)
testing_data_all_info.to_csv("testing_data.csv",index=False)

In [129]:
mb_size = 40

In [39]:
def custom_training_data_generator(training_data_all_info, multi_hot_encoded_Y_train, mb_size):
    
    idx = list(training_data_all_info.index)
    np.random.shuffle(idx)
    training_data_all_info = training_data_all_info.iloc[idx]
    multi_hot_encoded_Y_train = multi_hot_encoded_Y_train[idx]
    
    for time_step in range(training_data_all_info.shape[0]//mb_size):
        
        X_train_mb = list()
        
        for single_img_path in training_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size]["Image Path"]:
            
            resized_single_img =  cv2.resize(np.array(Image.open(single_img_path).convert("RGB")),(1024,1024))
            X_train_mb.append(resized_single_img)
            
        X_train_mb = np.array(X_train_mb)
        Y_train_mb = multi_hot_encoded_Y_train[time_step*mb_size:(time_step+1)*mb_size]
        
        yield X_train_mb, Y_train_mb

In [40]:
x, y = custom_training_data_generator(training_data_all_info, multi_hot_encoded_Y_train, mb_size).__next__()

In [41]:
y.shape

(40, 14)

In [42]:
testing_data_all_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h],Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,00013118_008.png,[Atelectasis],PA,2992,2991,0.143,0.143,/kaggle/input/data/images_006/images/00013118_...
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,00014716_007.png,"[Atelectasis, Effusion, Mass]",AP,3056,2544,0.139,0.139,/kaggle/input/data/images_007/images/00014716_...
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,00029817_009.png,[Atelectasis],AP,3056,2544,0.139,0.139,/kaggle/input/data/images_012/images/00029817_...
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,00014687_001.png,"[Atelectasis, Cardiomegaly, Consolidation]",AP,2500,2048,0.168,0.168,/kaggle/input/data/images_007/images/00014687_...
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,00017877_001.png,[Atelectasis],AP,2500,2048,0.168,0.168,/kaggle/input/data/images_008/images/00017877_...


In [ ]:
p = plt.imread(testing_data_all_info.iat[2, 13])
plt.imshow(p)

In [108]:
def custom_testing_data_generator(testing_data_all_info, multi_hot_encoded_Y_test, mb_size):
    
    for time_step in range(testing_data_all_info.shape[0]//mb_size):
        
        X_test_mb = list()
        
        for single_img_path in testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size]["Image Path"]:
            
            resized_single_img =  cv2.resize(np.array(Image.open(single_img_path).convert("RGB")),(1024,1024))
            X_test_mb.append(resized_single_img)
            
        X_test_mb = np.array(X_test_mb)
        #print(multi_hot_encoded_Y_test[time_step*mb_size:(time_step+1)*mb_size])
        Y_test_mb = multi_hot_encoded_Y_test[time_step*mb_size:(time_step+1)*mb_size]
        
        bbox_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,2:6])
        
        centroid_mb = bbox_mb[:,0:2] + 0.5*np.concatenate((bbox_mb[:,3:4],bbox_mb[:,2:3]),axis=1)
        
        img_size_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,9:11])
        
        ordered_img_size_mb = np.concatenate((img_size_mb[:,1:2],img_size_mb[:,0:1]),axis=1)
        
        rescaled_centroid_mb = centroid_mb * (32/ordered_img_size_mb)
        
        yield X_test_mb, Y_test_mb, rescaled_centroid_mb

In [111]:
testing_data_all_info = testing_data_all_info.iloc[0:5,]
multi_hot_encoded_Y_test = testing_data_all_info.iloc[0:5,]
x, y, z = custom_testing_data_generator(testing_data_all_info, multi_hot_encoded_Y_test, 1).__next__()

        Image Index Finding Label     Bbox [x           y          w  \
0  00013118_008.png   Atelectasis  225.084746  547.019217  86.779661   

          h]       Image Index Finding Labels View Position  \
0  79.186441  00013118_008.png  [Atelectasis]            PA   

   OriginalImage[Width  Height]  OriginalImagePixelSpacing[x     y]  \
0                 2992     2991                        0.143  0.143   

                                          Image Path  
0  /kaggle/input/data/images_006/images/00013118_...  


In [121]:
time_step = 0
mb_size = 3
type(multi_hot_encoded_Y_test[time_step*mb_size:(time_step+1)*mb_size])

pandas.core.frame.DataFrame

In [114]:
print(r)

NameError: name 'r' is not defined

In [ ]:
print(s)

In [ ]:
print(t)

In [ ]:
print(z)

In [ ]:
x.shape

In [ ]:
y.shape

In [46]:
z.shape

(40, 2)

In [47]:
class GlobalLSEPooling2D(tf.keras.layers.Layer):
    
    def __init__(self,r_hyper_param):
        super(GlobalLSEPooling2D, self).__init__()
        self.r = r_hyper_param
        
    def call(self,concatenated_input):
        x_star_per_channel = GlobalMaxPooling2D(keepdims=True)(concatenated_input)
        shifted_pix_values = tf.math.exp(self.r*(concatenated_input - x_star_per_channel))
        avged_output = GlobalAveragePooling2D(keepdims=True)(shifted_pix_values)
        logged_output = (1/self.r)*tf.math.log(avged_output)
        layer_output = x_star_per_channel + logged_output
        
        return layer_output

In [48]:
def chest_x_ray_cnn():
    input_to_cnn = Input(shape = (1024,1024,3))
    
    pretrained_vgg16_conv_base = VGG16(include_top=False,
                                    input_shape=(1024, 1024, 3),
                                    weights=None,
                                    classes=14)
    pretrained_vgg16_conv_base.load_weights('/kaggle/input/pretraineddata/pretraineddata/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
    pretrained_vgg16_conv_base.trainable = False

    pretrained_vgg19_conv_base = VGG19(include_top=False,
                                    input_shape=(1024, 1024, 3),
                                    weights=None,
                                    classes=14)
    pretrained_vgg19_conv_base.load_weights('/kaggle/input/pretraineddata/pretraineddata/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
    pretrained_vgg19_conv_base.trainable = False

    pretrained_resnet50_conv_base = ResNet50(include_top=False,
                                    input_shape=(1024, 1024, 3),
                                    weights=None,
                                    classes=14)
    pretrained_resnet50_conv_base.load_weights('/kaggle/input/pretraineddata/pretraineddata/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    pretrained_resnet50_conv_base.trainable = False

    pretrained_resnet101_conv_base = ResNet101(include_top=False,
                                    input_shape=(1024, 1024, 3),
                                    weights=None,
                                    classes=14)
    pretrained_resnet101_conv_base.load_weights('/kaggle/input/pretraineddata/pretraineddata/resnet101_weights_tf_dim_ordering_tf_kernels_notop.h5')
    pretrained_resnet101_conv_base.trainable = False
    
    pretrained_vgg16_conv_base.trainable = False
    pretrained_vgg19_conv_base.trainable = False
    pretrained_resnet50_conv_base.trainable = False
    pretrained_resnet101_conv_base.trainable = False
    
    vgg16_out = pretrained_vgg16_conv_base(input_to_cnn)
    vgg19_out = pretrained_vgg19_conv_base(input_to_cnn)
    resnet50_out = pretrained_resnet50_conv_base(input_to_cnn)
    resnet101_out = pretrained_resnet101_conv_base(input_to_cnn)
    
    concatenated_output = Concatenate()([vgg16_out,vgg19_out,
                                      resnet50_out,resnet101_out])
    
    pooled_output = GlobalLSEPooling2D(r_hyper_param=0.9)(concatenated_output)
    flattened_output = Flatten()(pooled_output)
    
    cnn_out = Dense(units=multi_hot_encoded_Y_train.shape[1],
                    activation="sigmoid")(flattened_output)
    
    return Model(inputs=input_to_cnn,outputs=cnn_out)

In [49]:
our_custom_cnn = chest_x_ray_cnn()

In [50]:
our_custom_cnn.summary

<bound method Model.summary of <keras.src.engine.functional.Functional object at 0x7984c5b232b0>>

In [51]:
def weighted_bcel(Y_train_mb,Y_pred_mb):
    
    cardinality_p = np.count_nonzero(Y_train_mb)
    cardinality_n = (Y_train_mb.shape[0]*Y_train_mb.shape[1]) - cardinality_p
    
    beta_p = (cardinality_p + cardinality_n + 10**(-7))/(cardinality_p + 10**(-7))
    beta_n = (cardinality_p + cardinality_n + 10**(-7))/(cardinality_n + 10**(-7))
    
    return -tf.reduce_mean(tf.reduce_mean(beta_p*Y_train_mb*tf.math.log(Y_pred_mb) \
                          + beta_n*(1-Y_train_mb)*tf.math.log(1-Y_pred_mb),axis=0))

In [53]:
def predicted_center_pixel_cords(heatmaps):
    
    argmax = list()
    for single_heatmap in heatmaps:
        
        heatmap_channels = cv2.split(single_heatmap)
        argmax_per_heatmap = list()
    
        for single_heatmap_channel in heatmap_channels:    
            single_channel_max_loc = list(np.unravel_index(np.argmax(single_heatmap_channel),single_heatmap_channel.shape))
            argmax_per_heatmap.append(single_channel_max_loc)
        
        argmax.append(np.array(argmax_per_heatmap)/np.array([[7,7]])*224)
        
    return np.array(argmax)

In [54]:
conv_base = Model(inputs=[our_custom_cnn.input],outputs=[our_custom_cnn.layers[5].output])

In [55]:
conv_base.summary

<bound method Model.summary of <keras.src.engine.functional.Functional object at 0x7984c5b23ca0>>

In [56]:
def heatmap_shape(h):
    h.shape

In [69]:
type(training_data_all_info)

pandas.core.frame.DataFrame

In [76]:
optimizer = tf.keras.optimizers.RMSprop()

num_epochs = 10
time_steps = 0
mb_size = 2
    #print('Running on TPU ', tpu.master())
for epoch in range(num_epochs):
    for X_train_mb,Y_train_mb in custom_training_data_generator(training_data_all_info.iloc[0:1,:],multi_hot_encoded_Y_train[0:1,:],mb_size = 1):
        
           #with tf.GradientTape() as tape:
            
            Y_pred_mb = our_custom_cnn(X_train_mb)
            w_bcel_value = weighted_bcel(Y_train_mb,Y_pred_mb)
            try:
                    gradients = tape.gradient(w_bcel_value,our_custom_cnn.trainable_weights)
                    optimizer.apply_gradients(zip(gradients,our_custom_cnn.trainable_weights))
            except:
                    print("None Object")
        
            train_acc,train_pre,train_rec = compute_performance_metrics(Y_pred_mb,Y_train_mb,0.5)
        
            time_steps += 1
            
            conv_base_out = conv_base(X_train_mb).numpy()
        #a = conv_base_out
            cls_head_params = our_custom_cnn.layers[-1].weights[0].numpy()
            #print(conv_base_out)
        #b = cls_head_params
            heatmaps = np.matmul(conv_base_out,cls_head_params)
        #val = heatmaps
        #print(heatmaps)
    
            pred_center_pix_loc = predicted_center_pixel_cords(heatmaps)
        
            print("Epoch # {}, Time Step # {}, Predicted Location of Each Disease for Chest X-rays in Mini Batch:".format(epoch,
                                                                                                                     time_steps))
            print(pred_center_pix_loc)

None Object
Epoch # 0, Time Step # 1, Predicted Location of Each Disease for Chest X-rays in Mini Batch:
[[[  0.   0.]
  [ 32. 928.]
  [448. 960.]
  [832. 928.]
  [128. 512.]
  [960.  32.]
  [736. 896.]
  [  0.   0.]
  [704. 160.]
  [ 96. 544.]
  [  0.   0.]
  [800.  64.]
  [  0.   0.]
  [256.  32.]]]
None Object
Epoch # 1, Time Step # 2, Predicted Location of Each Disease for Chest X-rays in Mini Batch:
[[[  0.   0.]
  [ 32. 928.]
  [448. 960.]
  [832. 928.]
  [128. 512.]
  [960.  32.]
  [736. 896.]
  [  0.   0.]
  [704. 160.]
  [ 96. 544.]
  [  0.   0.]
  [800.  64.]
  [  0.   0.]
  [256.  32.]]]
None Object
Epoch # 2, Time Step # 3, Predicted Location of Each Disease for Chest X-rays in Mini Batch:
[[[  0.   0.]
  [ 32. 928.]
  [448. 960.]
  [832. 928.]
  [128. 512.]
  [960.  32.]
  [736. 896.]
  [  0.   0.]
  [704. 160.]
  [ 96. 544.]
  [  0.   0.]
  [800.  64.]
  [  0.   0.]
  [256.  32.]]]
None Object
Epoch # 3, Time Step # 4, Predicted Location of Each Disease for Chest X-rays i

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
val.shape

In [ ]:
fold_size = 123
K = testing_data_all_info.shape[0]//fold_size
print(K)

In [77]:
fold_size = 123
K = testing_data_all_info.shape[0]//fold_size
print(K)

8


In [98]:
def compute_performance_metrics(Y_pred,Y_true,thresh):
    
    #print(Y_pred,' ', thresh)
    Y_pred = Y_pred > thresh
    #print(Y_true,' ',type(Y_true))
    confusion_matrix = multilabel_confusion_matrix(y_true=Y_true,y_pred=Y_pred)
    summed_confusion_matrix = np.sum(confusion_matrix,axis=0)
    
    tp = summed_confusion_matrix[0,0]
    tn = summed_confusion_matrix[1,1]
    fp = summed_confusion_matrix[0,1]
    fn = summed_confusion_matrix[1,0]
    
    
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    
    return accuracy,precision,recall    

In [130]:
type(multi_hot_encoded_Y_test)

numpy.ndarray

In [132]:
print("\n\n\n\n")

thresh_probability = np.arange(start=0.3,stop=0.8,step=0.1)
fold_size = 1
#K = testing_data_all_info.shape[0]//fold_size
K = 5
testing_data_all_info = testing_data_all_info.iloc[0:5,]
multi_hot_encoded_Y_test = multi_hot_encoded_Y_test[0:5,]
for fold_idx in range(1):
    for p_thresh in thresh_probability:
        
        cv_data = pd.concat([testing_data_all_info[0:(fold_idx*fold_size)],
                             testing_data_all_info[(fold_idx+1)*fold_size:]],axis=0)
        
        testing_data = testing_data_all_info[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        
        multi_hot_encoded_Y_cv = np.concatenate((multi_hot_encoded_Y_test[0:(fold_idx*fold_size)],
                                                multi_hot_encoded_Y_test[(fold_idx+1)*fold_size:]),
                                                axis=0)
        
        multi_hot_encoded_Y_test = multi_hot_encoded_Y_test[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        
        X_cv, Y_cv, rescaled_centroid_cv = custom_testing_data_generator(cv_data,multi_hot_encoded_Y_cv,
                                                                         cv_data.shape[0]).__next__()
        
        X_test, Y_test, rescaled_centroid_test = custom_testing_data_generator(testing_data,
                                                                               multi_hot_encoded_Y_test,
                                                                               testing_data.shape[0]).__next__()
    
        Y_pred_cv = our_custom_cnn(X_cv)
        
        #print(rescaled_centroid_cv)
        cv_acc,cv_pre,cv_rec = compute_performance_metrics(Y_pred_cv,Y_cv,p_thresh)
        
        cv_conv_base_out = conv_base(X_cv).numpy()
        cls_head_weights = our_custom_cnn.layers[-1].weights[0].numpy()
        cv_heatmaps = np.matmul(cv_conv_base_out,cls_head_weights)
     
        Y_pred_test = our_custom_cnn(X_test)
        test_acc,test_pre,test_rec = compute_performance_metrics(Y_pred_test,Y_test,p_thresh)
        
        test_conv_base_out = conv_base(X_test).numpy()
        test_heatmaps = np.matmul(test_conv_base_out,cls_head_weights)
        
        print("Threshold Probability: {}, CV Acc: {}, CV Prec: {}, CV Rec: {}".format(p_thresh,cv_acc,
                                                                                            cv_pre,
                                                                                            cv_rec))
        
        print("Threshold Probability: {}, Test Acc: {}, Test Prec: {}, Test Rec: {}".format(p_thresh,
                                                                                           test_acc,
                                                                                           test_pre,
                                                                                           test_rec))






[]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


ValueError: Found input variables with inconsistent numbers of samples: [0, 4]